# IRD counter 
#### Notebook to convert from labeled images to csv file with the IRD counts on depth

In [4]:
#imports
import skimage
from skimage import data,io,filters
from matplotlib import pyplot as plt
from matplotlib import ticker as plticker
import numpy as np
%matplotlib inline
import os
from skimage.measure import label
from skimage.morphology import opening,closing,erosion,dilation,disk,convex_hull_image
from skimage.measure import regionprops
from scipy.interpolate import interp1d
import pandas as pd
import math 

In [5]:
spliced=pd.read_csv('data/F23_U1537_reillysplice.csv')
section_df = pd.read_csv('data/J22_reillysupp_U1537_splice_sectionIDs_only.csv')
is_depth = pd.read_csv('data/interstitial_water_samples.csv')
section_df = section_df[section_df.Sect != 'CC']
section_df['Sect'] = section_df['Sect'].astype(str).astype(int)

In [6]:
spliced.rename(columns = {'Core Type':'Type'}, inplace = True)
spliced.rename(columns = {'Top Section':'Top section'}, inplace = True)
spliced.rename(columns = {'Bottom Section':'Bottom section'}, inplace = True)
spliced.rename(columns = {'Top Offset (cm)':'Top offset (cm)'}, inplace = True)
spliced.rename(columns = {'Bottom Offset (cm)':'Bottom offset (cm)'}, inplace = True)
spliced.head()

,Site,Hole,Core,Type,Top section,Top offset (cm),Top Depth (mbsf),Top Depth (mcd),Bottom section,Bottom offset (cm),...,Unnamed: 12,Splice Type,Comment,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,U1537,B,1,H,1,0.0,0.000,0.000,4,107.9,...,NaN,TIE,"Spliced Interval, as reported in the Expeditio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U1537,A,2,H,3,39.6,5.896,5.579,6,124.7,...,NaN,TIE,"Spliced Interval, as reported in the Expeditio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U1537,D,2,H,3,106.0,10.360,10.930,6,66.0,...,NaN,TIE,"Spliced Interval, as reported in the Expeditio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,U1537,A,3,H,1,136.9,13.369,15.030,6,27.2,...,NaN,TIE,"Spliced Interval, as reported in the Expeditio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U1537,D,3,H,4,9.8,20.398,21.433,5,123.9,...,NaN,TIE,"Spliced Interval, as reported in the Expeditio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def listdir(path):
    return list(listdir_gen(path))

def listdir_gen(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
            
def get_all_files_from_splice(n,rawpath):
    if not isinstance(n, list):
        n=[n]
    all_files = []
    for i in range(*n): 
        Hole = 'Hole '+spliced['Hole'].values[i]+'/'
        
        Core = spliced['Core'].values[i].astype(int).astype(str)+spliced['Type'].values[i]+'/'            
        Sections = np.arange(spliced['Top section'].values[i].astype(int),spliced['Bottom section'].values[i].astype(int)+1).astype(str)      
        files_this_row=np.concatenate([listdir(rawpath+Hole+Core+section) for section in Sections])
        all_files.append(files_this_row)
    return np.concatenate(all_files)

def picprep(image):
    whiteout = file.endswith('RAW.tif')
    if whiteout==True:
        image[0:image.shape[0], 0:100] = 65535
        return image
    else:
        return image

            
def generate_mask(image, width=200, train=False): 
    mask = picprep(image) 
    if width%2!=0:
        width=width-1
    mask = mask<mask.max()*.9
    mask = opening(mask,disk(5))
    mask = convex_hull_image(mask)
    middle_of_core=[]
    for y in range(mask.shape[0]):
        if np.sum(np.where(mask[y,:]))>0:
            middle_of_core.append(np.mean(np.where(mask[y,:])))
    middle_of_core = np.mean(middle_of_core).astype(int)
    if train==False:
        mask2 = np.zeros(mask.shape)
        for y in range(mask.shape[0]):
            if np.sum(mask[y,:])>0:
                left = np.array(middle_of_core-width/2).astype(int)
                right = np.array(middle_of_core+width/2).astype(int)
                mask2[y,left:right]=1
        return mask*mask2
    else:
        return mask
            
def IRDcounts(activefile,rawpath,labeled_path,width,spliced):
    Hole = 'Hole ' + activefile[9] + '/'
    Core = "".join(filter(str.isdigit, activefile[10:14])) + "".join(filter(str.isalpha, activefile[10:14])) + "/"
    Section = "".join(filter(str.isdigit, activefile[14:16])) + "/"
    
    rawfile=rawpath + Hole + Core + Section + activefile
    rawimage=skimage.io.imread(rawfile)

    image=skimage.io.imread(labeled_path + Hole + Core + Section + activefile)
    masks=np.zeros(image.shape[:2]) 
    core_mask = generate_mask(rawimage,train=False,width=width).astype(bool)
    masks[core_mask]=1

    mask=(image<.7)*masks
    maskclosed=closing(mask,disk(2))
    labeledimage=label(maskclosed,background=1)
    props=regionprops(labeledimage) 
    major_axis_length=[x.major_axis_length for x in props if x.major_axis_length < 500] 
    xs=[x.centroid[0] for x in props if x.major_axis_length < 500]
    xs_array=np.array(xs)
    true_top_in_pixels = 0
    true_bottom_in_pixels = labeledimage.shape[0]
    top=float(activefile.split("_")[1].split("-")[0])/10
    bottom=float(activefile.split("_")[1].split("-")[1])/10
    top_of_core = False
    if top==0:
        top=-2
        top_of_core=True
        
    mappedfunct=interp1d([true_top_in_pixels,true_bottom_in_pixels],[0,14],
                          fill_value="extrapolate",bounds_error=False) 
    depthcm=np.array(mappedfunct(xs_array))+top+1
    major_axis_length=np.array(major_axis_length)
    
    if not top_of_core:
        keep=depthcm>(top+3)
        major_axis_length = major_axis_length[keep]
        depthcm= depthcm[keep]
        
    round_depthcm = np.round(depthcm)
    all_ird = {'depth_cm': round_depthcm, 'IRDdia': major_axis_length}
    ird_df = pd.DataFrame(all_ird, columns = ["depth_cm","IRDdia"])
    bottom = ird_df.loc[(ird_df['depth_cm'] <= 148)] 
    depthcm = np.array(bottom['depth_cm'])
    major_axis_length = np.array(bottom['IRDdia'])
    ird_df = ird_df.loc[(ird_df['depth_cm'] <= 148)]
    ird_df = ird_df.loc[(ird_df['depth_cm'] >= 1)]
    ird_df = ird_df.loc[(ird_df['IRDdia'] >= 11)]
    ird_df['Hole'] = activefile[9]
    ird_df['Core'] = "".join(filter(str.isdigit, activefile[10:14]))
    ird_df['Section'] = "".join(filter(str.isdigit, activefile[14:16]))
    return ird_df

In [10]:
rawpath= '/local/data/EXP_382/U1537/'  
labeled_path = '/local/data/EXP_382/outputs_U1537/'
width=550
all_files = get_all_files_from_splice([0,1],rawpath)

In [11]:
all_files

array(['382-U1537B-1H-1-A_220-380_SHLF9937761_20190429160239_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_820-980_SHLF9937761_20190429160434_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_0-140_SHLF9937761_20190429160214_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_340-500_SHLF9937761_20190429160251_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_940-1100_SHLF9937761_20190429160447_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_1420-1500_SHLF9937761_20190429160537_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_580-740_SHLF9937761_20190429160409_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_100-260_SHLF9937761_20190429160226_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_460-620_SHLF9937761_20190429160304_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_700-860_SHLF9937761_20190429160422_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_1300-1460_SHLF9937761_20190429160525_k60_a1_t350_n20_RAW.tif',
       '382-U1537B-1H-1-A_1180-1340_SHLF

In [12]:
ird_df = pd.DataFrame([])
for file in all_files:
    data = IRDcounts(file,rawpath,labeled_path,width,spliced)
    ird_df = ird_df.append(data)

In [13]:
ird_df = ird_df.sort_values(by=['Hole','Core','Section','depth_cm'])
ird_df

,depth_cm,IRDdia,Hole,Core,Section
3,1.0,18.674248,B,1,1
6,3.0,31.553686,B,1,1
8,4.0,12.622622,B,1,1
9,4.0,34.857552,B,1,1
14,6.0,29.577244,B,1,1
...,...,...,...,...,...
22,148.0,14.821486,B,1,4
23,148.0,47.539781,B,1,4
25,148.0,12.259846,B,1,4
26,148.0,31.337027,B,1,4


In [14]:
# make a "core id" to join sections to ccsf
section_df['Hole'] = section_df['Hole'].astype('str')
core_id = section_df['Hole'] + '-' + section_df['Core'].astype(str) + '-' + section_df['Sect'].astype(str)
sections = {'core_id' : core_id,
           'CCSF_topsect_m' : section_df['Top depth CCSF 382 U1537 ABCD 20190504 USE THIS  m '] }
sect_df = pd.DataFrame(data = sections)
# make the same "core id" in IRD df to join sections df to ird df
ird_df['core_id'] = ird_df['Hole'] + '-' + ird_df['Core'].astype(str) + '-' + ird_df['Section'].astype(str) 
# join ird df to section df via core id 
ird_df = ird_df.join(sect_df.set_index('core_id'), on = 'core_id')
# add section length to CCSF top depth sect (m) to get CCSF for each grain 
ird_df['ccsf_cm'] = ird_df['depth_cm'] + ird_df['CCSF_topsect_m']*100
# convert ccsf from cm to meters
ird_df['CCSF_m'] = ird_df['ccsf_cm'] / 100
# sort the ird grains by depth (rather than by Hole)
ird_df = ird_df.sort_values(by=['CCSF_m'])

In [15]:
# gaps in data due to interstitial water samples (~5cm of cores missing)
intwat = is_depth['Hole'] + '-' + is_depth['Core'].astype(str) + '-' + is_depth['Sect'].astype(str)
data1 = {'core_id' : intwat,
           'iw_top_offset_cm' : is_depth['Top offset (cm)']}
intwat_df = pd.DataFrame(data = data1)
# join interstitial water with ird df 
ird_df = ird_df.join(intwat_df.set_index('core_id'), on = 'core_id')

ird_df['iw_top_offset_cm'] = ird_df['iw_top_offset_cm'].fillna(150)
ird_df['iw_minus1'] = ird_df['iw_top_offset_cm'].apply(lambda x: x-1 if x < 150 else x)
test_ird_df = ird_df[ird_df.depth_cm < ird_df.iw_minus1]
test_ird_df = test_ird_df.sort_values(by=['CCSF_m'])

In [16]:
# get rid of data before top offset and after bottom offset
top_id = spliced['Hole'] + '-' + spliced['Core'].astype(str) + '-' + spliced['Top section'].astype(str)
bottom_id = spliced['Hole'] + '-' + spliced['Core'].astype(str) + '-' + spliced['Bottom section'].astype(str)

top_offset = {'core_id' : top_id, 'top_offset_cm' : spliced['Top offset (cm)']}
top_offset_df = pd.DataFrame(data = top_offset)
toff_ird_df = test_ird_df.join(top_offset_df.set_index('core_id'), on = 'core_id')
toff_ird_df['top_offset_cm'] = toff_ird_df['top_offset_cm'].fillna(0)
test_toff_ird_df = toff_ird_df[toff_ird_df.depth_cm
                               >= toff_ird_df.top_offset_cm]


In [17]:
# adding in additional top offsets because some cores on reilly splice start at sect 1, 0 cm 
newoff = pd.read_csv('data/J22_U1537_newoffsets.csv')
extratops = newoff['Hole'] + '-' + newoff['Core'].astype(str) + '-' + newoff['Top Section'].astype(str)
add_top_offset = {'core_id' : extratops, 'new_top_offset_cm' : newoff['NEW Top Offset (cm)']}
add_top_offset_df = pd.DataFrame(data = add_top_offset)
extra_off = test_toff_ird_df.join(add_top_offset_df.set_index('core_id'), on = 'core_id') 
extra_off['new_top_offset_cm'] = extra_off['new_top_offset_cm'].fillna(0)
test_toff_ird_df = extra_off[extra_off.depth_cm
                               >= extra_off.new_top_offset_cm]

In [18]:
# remove data after bottom offset
bottom_id = spliced['Hole'] + '-' + spliced['Core'].astype(str) + '-' + spliced['Bottom section'].astype(str)
bot_offset = {'core_id' : bottom_id, 'bottom_offset_cm' : spliced['Bottom offset (cm)']}
bottom_offset_df = pd.DataFrame(data = bot_offset)
test_toff_ird_df = test_toff_ird_df.join(bottom_offset_df.set_index('core_id'), 
                                         on = 'core_id')
test_toff_ird_df['bottom_offset_cm'] = test_toff_ird_df['bottom_offset_cm'].fillna(150)
tboff_ird_df = test_toff_ird_df[test_toff_ird_df.depth_cm 
                                <= test_toff_ird_df.bottom_offset_cm]


In [19]:
# round the IRD m CCSF to 2 decimal points
test_round = tboff_ird_df.round({'CCSF_m' : 2})
curated = pd.read_csv('data/curated_length.csv')
cur_id = curated['Hole'] + '-' + curated['Core'].astype(str) + '-' + curated['Sect'].astype(str)
curate = {'core_id' : cur_id, 'curated len min 1' : curated['curated len min 1']}
curated_df = pd.DataFrame(data = curate)
cur_df = test_round.join(curated_df.set_index('core_id'), 
                                         on = 'core_id')
df = cur_df[cur_df['depth_cm'] <= cur_df['curated len min 1']]

In [25]:
df = df.drop(columns = ['depth_cm','CCSF_topsect_m','iw_top_offset_cm', 'iw_minus1',
        'top_offset_cm', 'new_top_offset_cm', 'bottom_offset_cm',
        'curated len min 1'])
df

,IRDdia,Hole,Core,Section,core_id,ccsf_cm,CCSF_m
3,18.674248,B,1,1,B-1-1,1.0,0.01
6,31.553686,B,1,1,B-1-1,3.0,0.03
8,12.622622,B,1,1,B-1-1,4.0,0.04
9,34.857552,B,1,1,B-1-1,4.0,0.04
14,29.577244,B,1,1,B-1-1,6.0,0.06
...,...,...,...,...,...,...,...
6,17.761355,B,1,4,B-1-4,549.0,5.49
17,11.215802,B,1,4,B-1-4,553.0,5.53
20,11.823441,B,1,4,B-1-4,553.0,5.53
16,20.718036,B,1,4,B-1-4,553.0,5.53


In [ ]:
df.to_csv('121923_U1537_cnn.csv')